In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv,DataFrame,concat
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('../input/cryptocurrencypricehistory/coin_Bitcoin.csv')


In [ ]:
#loop that turns time series into four lagging values + current value 
# df.iloc[[2]]
# for i in range(4,len(df)-5) 
#     take the four number before the date
#     add concat before number (make this a row)
#     make rows out every date in dataset 


In [ ]:
df_seq = pd.DataFrame(columns=["lag1", "lag2","lag3", "lag4","value"])

for i in range(4,len(df)-5):
    df_seq = df_seq.append({
         "lag1": df.iloc[[i-4],7,].values,
         "lag2": df.iloc[[i-3],7].values,
         "lag3": df.iloc[[i-2],7].values,
         "lag4": df.iloc[[i-1],7].values,
        "value": df.iloc[[i],7].values
    }, ignore_index=True)
df_seq = df_seq.apply(lambda row: row.astype(int),axis=1)

ss = StandardScaler()


In [ ]:

X_train,X_test,y_train,y_test = train_test_split(df_seq[["lag1", "lag2","lag3", "lag4"]],df_seq['value'],test_size=0.4,shuffle=False)

X_train = pd.DataFrame(ss.fit_transform(X_train),columns = ["lag1", "lag2","lag3", "lag4"])
X_test = pd.DataFrame(ss.transform(X_test),columns = ["lag1", "lag2","lag3", "lag4"])

y_train = pd.DataFrame(ss.fit_transform(np.array(y_train).reshape(-1,1)),columns =['value'])
y_test = pd.DataFrame(ss.transform(np.array(y_test).reshape(-1,1)),columns =['value'])

X_train =X_train.values.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test=X_test.values.reshape(X_test.shape[0],X_test.shape[1] , 1)

print(X_train.shape), print(X_test.shape)

In [ ]:
model=Sequential()
model.add(LSTM(100,return_sequences=True,input_shape=(4,1)))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(10))

model.add(Dense(1))
opt = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error',optimizer=opt)

In [ ]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=1,verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_test=ss.inverse_transform(y_test)
test_predict=model.predict(X_test)
test_predict = ss.inverse_transform(test_predict)


In [ ]:
plt.plot(range(1789,2982,1), test_predict, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(range(1789,2982,1), y_test, color='red', label='Actual Price')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.xticks(np.arange(1789,2982,500), df.Date[1789:2982:500])
plt.legend()
plt.show()